# Little Washita Spinup Example
This workbook illustrates how to spinup a watershed model. Spinup is the process of getting to a stable groundwater configuration in your model.  

There is no single best practice for spinup, results and approaches will vary depending on your domain and the questions you want to answer with your model. The goal is to have a domain that is stable (metrics for this may also vary) and solving nicely before you start making runs to answer questions

Groundwater is the slowest moving part so its often easiest to start with a simplified system and get a stable water table before adding in land surface processes. 

In this example we will apply a constant flux at the top of the model tha represents the long term recharge for our basin. We will run for a very long period of time, first with surface water turned off to make the problem easier to solve and then with surface water turned on.  We will use growth time steps so that we can take small steps at first when our problem is harder to solve and take progressively larger time steps as the model converges and gets easier to solve. 

We provide comments here on the steps that are unique to a spinup simulation. For a more thoroughly documented script refer to the [Little Washita Annoated ParFlow-CLM simulation script](https://github.com/hydroframe/parflow_python_shortcourse/blob/main/exercises/little_washita/LittleWashita_ParFlowCLM_AnnotatedExample%20copy.ipynb). 

For additional resources on ParFlow Keys and input scripts refer to the [ParFlow Manual](https://parflow.readthedocs.io/en/latest/keys.html#)

**Requriements** 
Before you can run this workbook you will need to make sure that you have parflow and pftools installed. Refer to the main readme of this repo for instructions on getting your modeling environment setup. if you haven't do so already.

**Simulation Inputs**
All input files can be found in the `model_inputs` folder. Here we will be using:
1. Solid file: used to define the shape of the domain 
2. Indicator file: used to define where the different geologic units are in the grid
3. Slope files: slope x and slope y files define the topographic slopes of each grid cell in the x and y directions

### 1. Import required libraries and functions 

In [ ]:
import os
import numpy as np
from parflow import Run 
import shutil
from parflow.tools.fs import mkdir, cp, get_absolute_path, exists
from parflow.tools.settings import set_working_directory

## 2. Setup your run directory and initialize your model object
Note we are are just copying in the slope files and the solid file. We don't need the CLM inputs, presure file or indicator file for this run. 

In [ ]:
# Name your ParFLow run -- note that all of your output files will have this prefix
runname = 'LW_Spinup'

# Create a directory in the outputs folder for this run
run_dir = get_absolute_path(f'outputs/{runname}')
mkdir(run_dir)
print(run_dir)

# create your Parflow model object. For starters we are just goin to set the file version and the run directory we'll add more later
# note that the model will run from the run_dir so all input files should be in the run dir or paths should be specified relative to run_dir
model = Run(runname, run_dir)
model.FileVersion = 4

#copy the model inputs for the simulation into the run directory
#NOTE: you dont have to copy everything into the run directory if you don't want, you can also point to input files in other directories in a simulation if you prefer
input_dir= os.path.join(os.getcwd(), 'model_inputs')
files = ['slopex_LW.pfb', 'slopey_LW.pfb', 'LW.pfsol', 'Indicator_LW_USGS_Bedrock.pfb']
for fname in files:
    shutil.copy(os.path.join(input_dir,fname), run_dir)

## 3. Setup the computational grid
We are going to go ahead and setup the grid they way we would when we are running full simulations here. Alghough the subsurface properties are not going to matter because we will make it all impermeable. If you prefer you can run your parking lot test with just one layer.

In [ ]:
# Processor topology: This is the way that the problem will be split across processors if you want to run in parallel
# The domain is divided in x,y and z dimensions by P, Q and R. The total number of processors is P*Q*R.
model.Process.Topology.P = 1
model.Process.Topology.Q = 1
model.Process.Topology.R = 1

#Locate the origin in the domain.
model.ComputationalGrid.Lower.X = 0.0
model.ComputationalGrid.Lower.Y = 0.0
model.ComputationalGrid.Lower.Z = 0.0

# Define the size of each grid cell. The length units are the same as those on hydraulic conductivity, here that is meters.
model.ComputationalGrid.DX = 1000.0
model.ComputationalGrid.DY = 1000.0
model.ComputationalGrid.DZ = 200.0

# Define the number of grid blocks in the domain.
model.ComputationalGrid.NX = 64
model.ComputationalGrid.NY = 32
model.ComputationalGrid.NZ = 10

### 3.2 Geometries

In our main simulations we willuse an indicator to define units in the subsurface. here we will make everything uniformly impermable so we just have a solid file input. 

In [ ]:
#Declare the geometries that you will use for the problem
model.GeomInput.Names = "solid_input indi_input"

#Define the solid_input geometry.  
#Note the naming convention here GeomInput.{GeomName}.key
model.GeomInput.solid_input.InputType = "SolidFile"
model.GeomInput.solid_input.GeomNames = "domain"
model.GeomInput.solid_input.FileName = "LW.pfsol"

#First set the name for your `Domain` and setup the patches for this domain
model.Domain.GeomName = "domain"
model.Geom.domain.Patches = "top bottom side"

# Next setup the indicator file geometry
model.GeomInput.indi_input.InputType =   "IndicatorField"
model.GeomInput.indi_input.GeomNames = "s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13 g1 g2 g3 g4 g5 g6 g7 g8 g9 g10"
model.Geom.indi_input.FileName = "Indicator_LW_USGS_Bedrock.pfb"

model.GeomInput.s1.Value =    1
model.GeomInput.s2.Value =    2
model.GeomInput.s3.Value =    3
model.GeomInput.s4.Value =    4
model.GeomInput.s5.Value =    5
model.GeomInput.s6.Value =    6
model.GeomInput.s7.Value =    7
model.GeomInput.s8.Value =    8
model.GeomInput.s9.Value =    9
model.GeomInput.s10.Value =   10
model.GeomInput.s11.Value =   11
model.GeomInput.s12.Value =   12
model.GeomInput.s13.Value =   13

model.GeomInput.g1.Value =    19
model.GeomInput.g2.Value =    20
model.GeomInput.g3.Value =    21
model.GeomInput.g4.Value =    22
model.GeomInput.g5.Value =    23
model.GeomInput.g6.Value =    24
model.GeomInput.g7.Value =    25
model.GeomInput.g8.Value =    26
model.GeomInput.g9.Value =    27
model.GeomInput.g10.Value =    28

### 3.3 Variable ${\Delta z}$ 
Here too we don't really need this for our test because we are focusing on the surface but we are keeping our full 3D grid so that we have it setup when we move to more complicated problems.

In [ ]:
model.Solver.Nonlinear.VariableDz = True
model.dzScale.GeomNames = "domain"
model.dzScale.Type = "nzList"
model.dzScale.nzListNumber = 10

model.Cell._0.dzScale.Value = 5
model.Cell._1.dzScale.Value = 0.5
model.Cell._2.dzScale.Value = 0.25
model.Cell._3.dzScale.Value = 0.125
model.Cell._4.dzScale.Value = 0.05
model.Cell._5.dzScale.Value = 0.025
model.Cell._6.dzScale.Value = 0.005
model.Cell._7.dzScale.Value = 0.003
model.Cell._8.dzScale.Value = 0.0015
model.Cell._9.dzScale.Value = 0.0005

### 3.4 Topographic slopes 
Next we define topographic slopes and values. This is the main input that we want to test with this script.  Slope files are derived from elevaiton maps and this processing, as well as the resolution of the underlying elevation map, can lead to local sinks and inconsitencies in the drainage network that are non-physical.  

In [ ]:
model.TopoSlopesX.Type = "PFBFile"
model.TopoSlopesX.GeomNames = "domain"
model.TopoSlopesX.FileName = "slopex_LW.pfb"

model.TopoSlopesY.Type = "PFBFile"
model.TopoSlopesY.GeomNames = "domain"
model.TopoSlopesY.FileName = "slopey_LW.pfb"

## 4. Setup the surface and subsurface properties 
We are going to setup the subsurface properties using the geometries from the indicator file we read in aabove. 

NOTE: you can save a lot of lines of python by reading these properties in as a table instead. Refer to the Little_Washita_TableExample.ipynb to see how to do this. 

### 4.1 Permeability

In [ ]:
model.Geom.Perm.Names = "domain s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13 g1 g2 g3 g4 g5 g6 g7 g8 g9 g10"

model.Geom.domain.Perm.Type = "Constant"
model.Geom.domain.Perm.Value = 0.02

model.Geom.s1.Perm.Type = "Constant"
model.Geom.s1.Perm.Value = 0.269022595

model.Geom.s2.Perm.Type = "Constant"
model.Geom.s2.Perm.Value = 0.043630356

model.Geom.s3.Perm.Type = "Constant"
model.Geom.s3.Perm.Value = 0.015841225

model.Geom.s4.Perm.Type = "Constant"
model.Geom.s4.Perm.Value = 0.007582087

model.Geom.s5.Perm.Type = "Constant"
model.Geom.s5.Perm.Value = 0.01818816

model.Geom.s6.Perm.Type = "Constant"
model.Geom.s6.Perm.Value = 0.005009435

model.Geom.s7.Perm.Type = "Constant"
model.Geom.s7.Perm.Value = 0.005492736

model.Geom.s8.Perm.Type = "Constant"
model.Geom.s8.Perm.Value = 0.004675077

model.Geom.s9.Perm.Type = "Constant"
model.Geom.s9.Perm.Value = 0.003386794

model.Geom.s10.Perm.Type = "Constant"
model.Geom.s10.Perm.Value = 0.004783973

model.Geom.s11.Perm.Type = "Constant"
model.Geom.s11.Perm.Value = 0.003979136

model.Geom.s12.Perm.Type = "Constant"
model.Geom.s12.Perm.Value = 0.006162952

model.Geom.s13.Perm.Type = "Constant"
model.Geom.s13.Perm.Value = 0.005009435

model.Geom.g1.Perm.Type = "Constant"
model.Geom.g1.Perm.Value = 5e-3

model.Geom.g2.Perm.Type = "Constant"
model.Geom.g2.Perm.Value = 1e-2

model.Geom.g3.Perm.Type = "Constant"
model.Geom.g3.Perm.Value = 2e-2

model.Geom.g4.Perm.Type = "Constant"
model.Geom.g4.Perm.Value = 3e-2

model.Geom.g5.Perm.Type = "Constant"
model.Geom.g5.Perm.Value = 4e-2

model.Geom.g6.Perm.Type = "Constant"
model.Geom.g6.Perm.Value = 5e-2

model.Geom.g7.Perm.Type = "Constant"
model.Geom.g7.Perm.Value = 6e-2

model.Geom.g8.Perm.Type = "Constant"
model.Geom.g8.Perm.Value = 8e-2

model.Geom.g9.Perm.Type = "Constant"
model.Geom.g9.Perm.Value = 0.1

model.Geom.g10.Perm.Type = "Constant"
model.Geom.g10.Perm.Value = 0.2

### 4.2 Permeability tensor

In [ ]:
model.Perm.TensorType = "TensorByGeom"
model.Geom.Perm.TensorByGeom.Names = "domain"
model.Geom.domain.Perm.TensorValX = 1.0
model.Geom.domain.Perm.TensorValY = 1.0
model.Geom.domain.Perm.TensorValZ = 1.0

### 4.3  Specific Storage

In [ ]:
model.SpecificStorage.Type = "Constant"
model.SpecificStorage.GeomNames = "domain"
model.Geom.domain.SpecificStorage.Value = 0.0001

### 4.4 Porosity

In [ ]:
model.Geom.Porosity.GeomNames = "domain s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13"

model.Geom.domain.Porosity.Type = "Constant"
model.Geom.domain.Porosity.Value = 0.33

model.Geom.s1.Porosity.Type = "Constant"
model.Geom.s1.Porosity.Value = 0.375

model.Geom.s2.Porosity.Type = "Constant"
model.Geom.s2.Porosity.Value = 0.39

model.Geom.s3.Porosity.Type = "Constant"
model.Geom.s3.Porosity.Value = 0.387

model.Geom.s4.Porosity.Type = "Constant"
model.Geom.s4.Porosity.Value = 0.439

model.Geom.s5.Porosity.Type = "Constant"
model.Geom.s5.Porosity.Value = 0.489

model.Geom.s6.Porosity.Type = "Constant"
model.Geom.s6.Porosity.Value = 0.399

model.Geom.s7.Porosity.Type = "Constant"
model.Geom.s7.Porosity.Value = 0.384

model.Geom.s8.Porosity.Type = "Constant"
model.Geom.s8.Porosity.Value = 0.482

model.Geom.s9.Porosity.Type = "Constant"
model.Geom.s9.Porosity.Value = 0.442

model.Geom.s10.Porosity.Type = "Constant"
model.Geom.s10.Porosity.Value = 0.385

model.Geom.s11.Porosity.Type = "Constant"
model.Geom.s11.Porosity.Value = 0.481

model.Geom.s12.Porosity.Type = "Constant"
model.Geom.s12.Porosity.Value = 0.459

model.Geom.s13.Porosity.Type = "Constant"
model.Geom.s13.Porosity.Value = 0.399

### 4.5 Relative Permeability

In [ ]:
model.Phase.RelPerm.Type =              "VanGenuchten"
model.Phase.RelPerm.GeomNames =     "domain s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13"

model.Geom.domain.RelPerm.Alpha =    1.0
model.Geom.domain.RelPerm.N =        3.0

model.Geom.s1.RelPerm.Alpha =        3.548
model.Geom.s1.RelPerm.N =            4.162

model.Geom.s2.RelPerm.Alpha =        3.467
model.Geom.s2.RelPerm.N =            2.738

model.Geom.s3.RelPerm.Alpha =        2.692
model.Geom.s3.RelPerm.N =            2.445

model.Geom.s4.RelPerm.Alpha =        0.501
model.Geom.s4.RelPerm.N =            2.659

model.Geom.s5.RelPerm.Alpha =        0.661
model.Geom.s5.RelPerm.N =            2.659

model.Geom.s6.RelPerm.Alpha =        1.122
model.Geom.s6.RelPerm.N =            2.479

model.Geom.s7.RelPerm.Alpha =        2.089
model.Geom.s7.RelPerm.N =            2.318

model.Geom.s8.RelPerm.Alpha =        0.832
model.Geom.s8.RelPerm.N =            2.514

model.Geom.s9.RelPerm.Alpha =        1.585
model.Geom.s9.RelPerm.N =            2.413

model.Geom.s10.RelPerm.Alpha =        3.311
model.Geom.s10.RelPerm.N =            2.202

model.Geom.s11.RelPerm.Alpha =        1.622
model.Geom.s11.RelPerm.N =            2.318

model.Geom.s12.RelPerm.Alpha =        1.514
model.Geom.s12.RelPerm.N =            2.259

model.Geom.s13.RelPerm.Alpha =        1.122
model.Geom.s13.RelPerm.N =            2.479

### 4.6 Saturation

In [ ]:
model.Phase.Saturation.Type =             "VanGenuchten"
model.Phase.Saturation.GeomNames =         "domain s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13"

model.Geom.domain.Saturation.Alpha =        1.0
model.Geom.domain.Saturation.N =            3.0
model.Geom.domain.Saturation.SRes =         0.001
model.Geom.domain.Saturation.SSat =         1.0

model.Geom.s1.Saturation.Alpha =        3.548
model.Geom.s1.Saturation.N =            4.162
model.Geom.s1.Saturation.SRes =         0.0001
model.Geom.s1.Saturation.SSat =         1.0

model.Geom.s2.Saturation.Alpha =        3.467
model.Geom.s2.Saturation.N =            2.738
model.Geom.s2.Saturation.SRes =         0.0001
model.Geom.s2.Saturation.SSat =         1.0

model.Geom.s3.Saturation.Alpha =        2.692
model.Geom.s3.Saturation.N =            2.445
model.Geom.s3.Saturation.SRes =         0.0001
model.Geom.s3.Saturation.SSat =         1.0

model.Geom.s4.Saturation.Alpha =        0.501
model.Geom.s4.Saturation.N =            2.659
model.Geom.s4.Saturation.SRes =         0.1
model.Geom.s4.Saturation.SSat =         1.0

model.Geom.s5.Saturation.Alpha =        0.661
model.Geom.s5.Saturation.N =            2.659
model.Geom.s5.Saturation.SRes =         0.0001
model.Geom.s5.Saturation.SSat =         1.0

model.Geom.s6.Saturation.Alpha =        1.122
model.Geom.s6.Saturation.N =            2.479
model.Geom.s6.Saturation.SRes =         0.0001
model.Geom.s6.Saturation.SSat =         1.0

model.Geom.s7.Saturation.Alpha =        2.089
model.Geom.s7.Saturation.N =            2.318
model.Geom.s7.Saturation.SRes =         0.0001
model.Geom.s7.Saturation.SSat =         1.0

model.Geom.s8.Saturation.Alpha =        0.832
model.Geom.s8.Saturation.N =            2.514
model.Geom.s8.Saturation.SRes =         0.0001
model.Geom.s8.Saturation.SSat =         1.0

model.Geom.s9.Saturation.Alpha =        1.585
model.Geom.s9.Saturation.N =            2.413
model.Geom.s9.Saturation.SRes =         0.0001
model.Geom.s9.Saturation.SSat =         1.0

model.Geom.s10.Saturation.Alpha =        3.311
model.Geom.s10.Saturation.N =            2.202
model.Geom.s10.Saturation.SRes =         0.0001
model.Geom.s10.Saturation.SSat =         1.0

model.Geom.s11.Saturation.Alpha =        1.622
model.Geom.s11.Saturation.N =            2.318
model.Geom.s11.Saturation.SRes =         0.0001
model.Geom.s11.Saturation.SSat =         1.0

model.Geom.s12.Saturation.Alpha =        1.514
model.Geom.s12.Saturation.N =            2.259
model.Geom.s12.Saturation.SRes =         0.0001
model.Geom.s12.Saturation.SSat =         1.0

model.Geom.s13.Saturation.Alpha =        1.122
model.Geom.s13.Saturation.N =            2.479
model.Geom.s13.Saturation.SRes =         0.0001
model.Geom.s13.Saturation.SSat =         1.0

### 4.7 Manning's coefficient

In [ ]:
model.Mannings.Type = "Constant"
model.Mannings.GeomNames = "domain"
model.Mannings.Geom.domain.Value = 0.0000044

## 5. Phases contaminants, gravity and wells

In [ ]:
# Phases
model.Phase.Names = "water"
model.Phase.water.Density.Type = "Constant"
model.Phase.water.Density.Value = 1.0
model.Phase.water.Viscosity.Type = "Constant"
model.Phase.water.Viscosity.Value = 1.0
model.Phase.water.Mobility.Type = "Constant"
model.Phase.water.Mobility.Value = 1.0

# Contaminants
model.Contaminants.Names = ""

# Gravity
model.Gravity = 1.0

#Wells
model.Wells.Names = ""

# Phase Sources
model.PhaseSources.water.Type = "Constant"
model.PhaseSources.water.GeomNames = "domain"
model.PhaseSources.water.Geom.domain.Value = 0.0

## 6. Timing
The units of time are set by the *hydraulic conductivity*, $K$ units $[LT-1]$, in the case our units are *hours*. 

For spinup we are going to run our simulation for a very long time with a constant rechage forcing across the top boundray. 

**Run time:** Here we are setting our simulation to run for 10,000,000 hours writing outputs every 100 hours. 

**Growth Timestep:** When doing spinup simulations we generally use a growth timestep. This allows the model to solve smaller timesteps when its first starting out but to take progressively larger and larger steps as the simulation continues and *hopefully* your problem is converging and getting easier to solve.  In this case we will take an initial time step of 1 hour and the size of that time step will grow exponentially with an exponent of 1.1 up to the max time step size of 100. 

**Time Cycles:** Since none of our inputs will be changing with time we can just setup a single time cycle called '`constant` with one period called `alltime` that will repeat for the entire simulation (indicated by setting `repeat` to -1). 


In [ ]:
# Time units ans start and stop times
model.TimingInfo.BaseUnit = 1.0
model.TimingInfo.StartCount = 0
model.TimingInfo.StartTime = 0.0
model.TimingInfo.StopTime = 10000000.0
model.TimingInfo.DumpInterval = 100.0

# Growth timestep properties
model.TimeStep.Type = "Growth"
model.TimeStep.InitialStep = 1.0
model.TimeStep.GrowthFactor = 1.1
model.TimeStep.MaxStep = 100
model.TimeStep.MinStep = 1

#Time cycles
model.Cycle.Names ="constant"
model.Cycle.constant.Names = "alltime"
model.Cycle.constant.alltime.Length = 1
model.Cycle.constant.Repeat = -1

## 7. Boundary and intial conditions

### 7.1 Boundary conditions
Now, we assign Boundary Conditions for each face (each of the Patches in the domain defined before). Recall the previously stated Patches and associate them with the boundary conditions that follow. The bottom and sides of our domain are all set to no-flow (i.e. constant flux of 0) boundaries. 

For the top boundary we are going to apply a constant flux that will represent the long term average recharge of the domain. By running with this constant flux for a long period of time we can acheive a steady state groundwater configuraiton that will be the starting point for our later transient simulations. 

Here we use a negative flux because the z-axis in ParFlow points up so a negative value indicates a downward flux into the top of our model. 

NOTE1: The units of this flux should be [1/T] so if you have a flux that is [L/T] remember to divide by the thickness of the layer you are applying it to (in this case our top layer).

NOTE2: If you don't want to apply the same forcing everywhere in your domain you can use a pfb file to setup a spatially variable flux and read it in like this:
```
model.Solver.EvapTransFile  = True
model.Solver.EvapTrans.FileName = “PmE.flux.pfb“
```
The PME file you provide should be a 3D file with the same number of layers as your domain. If you just want to apply a flux to the top boundary you can have the values in all the other layers set to 0. Also don't forget if you read this file in you will need to add it to the list of files you distribute before you run your model. 

In [ ]:
model.BCPressure.PatchNames = "top bottom side"

model.Patch.bottom.BCPressure.Type = "FluxConst"
model.Patch.bottom.BCPressure.Cycle = "constant"
model.Patch.bottom.BCPressure.alltime.Value = 0.0

model.Patch.side.BCPressure.Type = "FluxConst"
model.Patch.side.BCPressure.Cycle = "constant"
model.Patch.side.BCPressure.alltime.Value = 0.0

model.Patch.top.BCPressure.Type = "OverlandKinematic"
model.Patch.top.BCPressure.Cycle = "constant"
model.Patch.top.BCPressure.alltime.Value = -2.1e-5

### 7.2 Initial conditions: water pressure
We are going to start our model completely dry by indicating a pressure value of 0 relative to our bottom patch.  Note that this is a subjective choice. Depending on your domain you might converge to your steady groundwater configuration faster by starting your domain completely full or something in the middle.  

NOTE 1: The steady state solution you converge to at the end of spinup should be the same regardless of your initial conditions. Changing the initial conditions will just change the number of timesteps it takes your model to get there. 

NOTE 2: Spinups can take a long time and you may end up needing to change settings at some point and restart your model. You don't have to start from scratch every time though. You can pick up where you left off by setting your initial conditions to the last pressure file output you generated.  An example of that is included in the commented block of code below. You would just want to replace `press.init.pfb` with the name of the pressure file you are wanting to start from and make sure to put that pressure file into your run directory. 

In [ ]:
# Starting from a constant head values
model.ICPressure.Type = "HydroStaticPatch"
model.ICPressure.GeomNames = "domain"
model.Geom.domain.ICPressure.Value = 0.0
model.Geom.domain.ICPressure.RefGeom = "domain"
model.Geom.domain.ICPressure.RefPatch = "bottom"

#Starting from a previous simulation output
#model.ICPressure.Type = "PFBFile"
#model.ICPressure.GeomNames = "domain"
#model.Geom.domain.ICPressure.RefPatch = "top"
#model.Geom.domain.ICPressure.FileName = "press.init.pfb"

## 8. Solver settings

### 8.1 Outputs:
Now we specify what outputs we would like written. In this example we specify that we would like to write out CLM variables as well as Pressure and Saturation. However, there are many options for this and you should change these options according to what type of analysis you will be performing on your results. A complete list of print options is provided in § 6.1.32.

In [ ]:
model.Solver.PrintSubsurfData = True
model.Solver.PrintPressure = True
model.Solver.PrintSaturation = True
model.Solver.PrintMask = True
model.Solver.PrintVelocities = False

### 8.2 General Solver parameters

In [ ]:
# Solver types
model.Solver = "Richards"
model.Solver.TerrainFollowingGrid = True
model.Solver.Linear.Preconditioner = "PFMG"
model.Solver.Linear.Preconditioner.PCMatrixType = "FullJacobian"

# Exact solution
model.KnownSolution = "NoKnownSolution"

# Solver settings
model.Solver.MaxIter = 25000
model.Solver.Drop = 1e-20
model.Solver.AbsTol = 1e-8

model.Solver.MaxConvergenceFailures = 8
model.Solver.Nonlinear.MaxIter = 1000
model.Solver.Nonlinear.ResidualTol = 1e-6
model.Solver.Nonlinear.EtaChoice =  "EtaConstant"
model.Solver.Nonlinear.EtaValue = 0.001
model.Solver.Nonlinear.UseJacobian = True
model.Solver.Nonlinear.DerivativeEpsilon = 1e-16
model.Solver.Nonlinear.StepTol = 1e-15
model.Solver.Nonlinear.Globalization = "LineSearch"


model.Solver.Linear.KrylovDimension = 70
model.Solver.Linear.MaxRestarts = 2

### 8.3 Solver settings for spinup
There are a couple of tricks we can play to get the model to solve faster when we are in spinning it up. 

**OverlandFlowSpinUp Key:** When it is set to one this key removes any ponded surface water at every time step so that no overland flow occurs. Essentially when this is turned on you have shut off overland flow making the problem easier to solve. Note that this is just a tool to make the problem easier to solve in the beginning and *should not be used for regular simulation*. 

After you have a model that is well converged with this key turned on you will want to turn this back off by commenting out the line or seting the value to 0 and continuing to run until your streams have formed and you have a steady solution. 

When you do this make sure that you use the last pressure file from your previous simulation as your initial condition or you will be starting from scratch. 

In [ ]:
model.Solver.OverlandFlowSpinUp = 1

## 9.  Distribute files, write the model and run the simulation

In [ ]:
#distribute input files
#slope files are 2D (i.e. they only have one layer) so you need to set NZ to 1 before you distribute them
#Make sure to set it back to your actual NZ before distributing 3D files or running your model
model.ComputationalGrid.NZ =1
model.dist("slopex_LW.pfb")
model.dist("slopey_LW.pfb")

model.ComputationalGrid.NZ = 10 #the rest of the inputs should be distributed over 3D space
model.dist("Indicator_LW_USGS_Bedrock.pfb")

# write
model.write()
model.write(file_format='yaml')
model.write(file_format='json')

#run
model.run()